Task-16.1

In [6]:
import sqlite3

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create Books table
cursor.execute('''
CREATE TABLE Books (
    BookID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title TEXT NOT NULL,
    Author TEXT NOT NULL,
    ISBN TEXT UNIQUE NOT NULL,
    PublicationYear INTEGER,
    Status TEXT NOT NULL DEFAULT 'Available'
);
''')

# Create Members table
cursor.execute('''
CREATE TABLE Members (
    MemberID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Address TEXT,
    PhoneNumber TEXT,
    Email TEXT UNIQUE
);
''')

# Create Loans table
cursor.execute('''
CREATE TABLE Loans (
    LoanID INTEGER PRIMARY KEY AUTOINCREMENT,
    BookID INTEGER NOT NULL,
    MemberID INTEGER NOT NULL,
    LoanDate TEXT NOT NULL,
    ReturnDate TEXT,
    FOREIGN KEY (BookID) REFERENCES Books(BookID),
    FOREIGN KEY (MemberID) REFERENCES Members(MemberID)
);
''')

# Commit the changes
conn.commit()

print("Database schema created successfully with Books, Members, and Loans tables.")


# --- Code to display schema ---
# Function to display table schema using PRAGMA table_info
def display_table_schema(table_name):
    print(f"\n--- Schema for {table_name} table ---")
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema_info = cursor.fetchall()
    if not schema_info:
        print(f"No schema information found for {table_name}.")
        return

    # Print header
    print(f"{'CID':<4} {'Name':<15} {'Type':<10} {'NotNull':<8} {'PK':<3} {'Dflt_Value':<15}")
    print(f"{'----':<4} {'---------------':<15} {'----------':<10} {'--------':<8} {'---':<3} {'---------------':<15}")

    # Print rows
    for col in schema_info:
        # col format: (cid, name, type, notnull, dflt_value, pk)
        print(f"{col[0]:<4} {col[1]:<15} {col[2]:<10} {bool(col[3]):<8} {bool(col[5]):<3} {str(col[4]):<15}")

# Display schema for each table
display_table_schema('Books')
display_table_schema('Members')
display_table_schema('Loans')

Database schema created successfully with Books, Members, and Loans tables.

--- Schema for Books table ---
CID  Name            Type       NotNull  PK  Dflt_Value     
---- --------------- ---------- -------- --- ---------------
0    BookID          INTEGER    0        1   None           
1    Title           TEXT       1        0   None           
2    Author          TEXT       1        0   None           
3    ISBN            TEXT       1        0   None           
4    PublicationYear INTEGER    0        0   None           
5    Status          TEXT       1        0   'Available'    

--- Schema for Members table ---
CID  Name            Type       NotNull  PK  Dflt_Value     
---- --------------- ---------- -------- --- ---------------
0    MemberID        INTEGER    0        1   None           
1    Name            TEXT       1        0   None           
2    Address         TEXT       0        0   None           
3    PhoneNumber     TEXT       0        0   None           
4   

Task -16.2

In [7]:
# --- Insert Sample Books ---
print("\n--- Inserting Sample Books ---")
books_data = [
    ('The Great Gatsby', 'F. Scott Fitzgerald', '978-0743273565', 1925, 'Available'),
    ('1984', 'George Orwell', '978-0451524935', 1949, 'Available'),
    ('To Kill a a Mockingbird', 'Harper Lee', '978-0061120084', 1960, 'Available')
]
cursor.executemany("INSERT INTO Books (Title, Author, ISBN, PublicationYear, Status) VALUES (?, ?, ?, ?, ?)", books_data)
conn.commit()
print("3 Books inserted.")

# --- Insert Sample Members ---
print("\n--- Inserting Sample Members ---")
members_data = [
    ('Alice Smith', '123 Main St', '555-1111', 'alice@example.com'),
    ('Bob Johnson', '456 Oak Ave', '555-2222', 'bob@example.com'),
    ('Charlie Brown', '789 Pine Ln', '555-3333', 'charlie@example.com')
]
cursor.executemany("INSERT INTO Members (Name, Address, PhoneNumber, Email) VALUES (?, ?, ?, ?)", members_data)
conn.commit()
print("3 Members inserted.")

# --- Insert Sample Loans ---
print("\n--- Inserting Sample Loans ---")
# Assuming BookID 1, 2, 3 and MemberID 1, 2, 3 were auto-generated
loans_data = [
    (1, 1, '2023-01-01', None), # Alice borrowed The Great Gatsby
    (2, 2, '2023-01-05', None), # Bob borrowed 1984
    (3, 3, '2023-01-10', '2023-01-25') # Charlie borrowed To Kill a Mockingbird and returned it
]
cursor.executemany("INSERT INTO Loans (BookID, MemberID, LoanDate, ReturnDate) VALUES (?, ?, ?, ?)", loans_data)
conn.commit()
print("3 Loans inserted.")

# --- Display Inserted Rows ---

def display_table_data(table_name):
    print(f"\n--- Data in {table_name} table ---")
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    if not rows:
        print(f"No data in {table_name}.")
        return

    # Get column names for header
    col_names = [description[0] for description in cursor.description]
    print(" | ".join(col_names))
    print("-" * (sum(len(col) for col in col_names) + 3 * (len(col_names) - 1)))

    for row in rows:
        print(" | ".join(map(str, row)))


display_table_data('Books')
display_table_data('Members')
display_table_data('Loans')



--- Inserting Sample Books ---
3 Books inserted.

--- Inserting Sample Members ---
3 Members inserted.

--- Inserting Sample Loans ---
3 Loans inserted.

--- Data in Books table ---
BookID | Title | Author | ISBN | PublicationYear | Status
---------------------------------------------------------
1 | The Great Gatsby | F. Scott Fitzgerald | 978-0743273565 | 1925 | Available
2 | 1984 | George Orwell | 978-0451524935 | 1949 | Available
3 | To Kill a a Mockingbird | Harper Lee | 978-0061120084 | 1960 | Available

--- Data in Members table ---
MemberID | Name | Address | PhoneNumber | Email
-----------------------------------------------
1 | Alice Smith | 123 Main St | 555-1111 | alice@example.com
2 | Bob Johnson | 456 Oak Ave | 555-2222 | bob@example.com
3 | Charlie Brown | 789 Pine Ln | 555-3333 | charlie@example.com

--- Data in Loans table ---
LoanID | BookID | MemberID | LoanDate | ReturnDate
--------------------------------------------------
1 | 1 | 1 | 2023-01-01 | None
2 | 2 | 2 |

Task-16.3

In [8]:
print("\n--- Loan Details ---")

# Construct the SQL query with JOINs
query = """
SELECT
    B.Title,
    M.Name AS MemberName,
    L.LoanDate,
    L.ReturnDate
FROM
    Loans AS L
JOIN
    Books AS B ON L.BookID = B.BookID
JOIN
    Members AS M ON L.MemberID = M.MemberID
"""

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Get column names for header
column_names = [description[0] for description in cursor.description]

# Display results in a readable table format
if not results:
    print("No loan data found.")
else:
    # Print header
    print(" | ".join(column_names))
    print("-" * (sum(len(col) for col in column_names) + 3 * (len(column_names) - 1)))

    # Print rows
    for row in results:
        print(" | ".join(map(str, row)))


--- Loan Details ---
Title | MemberName | LoanDate | ReturnDate
------------------------------------------
The Great Gatsby | Alice Smith | 2023-01-01 | None
1984 | Bob Johnson | 2023-01-05 | None
To Kill a a Mockingbird | Charlie Brown | 2023-01-10 | 2023-01-25
